In [ ]:
# Setup - Load OpenVINO from local build
import sys
import os
from pathlib import Path

# Add local OpenVINO to path
ov_bin = Path(r"bin\intel64\Release\python")
if ov_bin.exists():
    sys.path.insert(0, str(ov_bin.resolve()))

import openvino as ov
import numpy as np
import matplotlib.pyplot as plt
import json
from collections import Counter

print(f"✅ OpenVINO: {ov.__version__}")

In [ ]:
# Load models (just graph structure, no compilation!)
core = ov.Core()

original = core.read_model("gemma_ir_tssn/openvino_model.xml")
evolved = core.read_model("gemma_ir_tssn/evolved_checkpoint.xml")

print(f"📦 Original: {len(list(original.get_ops()))} ops")
print(f"🧬 Evolved: {len(list(evolved.get_ops()))} ops")

In [ ]:
# Extract TSSN function_ids from both models
def get_tssn_function_ids(model):
    tssn_layers = {}
    for op in model.get_ops():
        if op.get_type_name() == "TSSN":
            layer_name = op.get_friendly_name()
            attrs = op.get_attributes()
            function_ids = attrs.get('function_ids', '').split(',')
            function_ids = [int(f) for f in function_ids if f]
            tssn_layers[layer_name] = function_ids
    return tssn_layers

original_ids = get_tssn_function_ids(original)
evolved_ids = get_tssn_function_ids(evolved)

print(f"🎯 Found {len(original_ids)} TSSN layers")

In [ ]:
# Calculate mutation statistics per layer
mutation_stats = []

for layer_name in sorted(original_ids.keys()):
    orig = np.array(original_ids[layer_name])
    evol = np.array(evolved_ids[layer_name])
    
    changes = np.sum(orig != evol)
    pct = (changes / len(orig)) * 100
    
    mutation_stats.append({
        'layer': layer_name,
        'total_neurons': len(orig),
        'mutations': changes,
        'mutation_pct': pct
    })

# Display as table (Data Wrangler will make this interactive!)
import pandas as pd
df = pd.DataFrame(mutation_stats)
df

In [ ]:
# Visualize mutation distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar chart of mutations per layer
axes[0].bar(range(len(df)), df['mutation_pct'])
axes[0].set_xlabel('Layer Index')
axes[0].set_ylabel('Mutation %')
axes[0].set_title('Mutation Rate by Layer')
axes[0].axhline(y=df['mutation_pct'].mean(), color='r', linestyle='--', label=f"Avg: {df['mutation_pct'].mean():.1f}%")
axes[0].legend()

# Histogram of mutation percentages
axes[1].hist(df['mutation_pct'], bins=15, edgecolor='black')
axes[1].set_xlabel('Mutation %')
axes[1].set_ylabel('Layer Count')
axes[1].set_title('Distribution of Mutation Rates')

plt.tight_layout()
plt.show()

print(f"\n📊 Summary:")
print(f"  Total Mutations: {df['mutations'].sum():,} / {df['total_neurons'].sum():,}")
print(f"  Overall Rate: {(df['mutations'].sum() / df['total_neurons'].sum() * 100):.2f}%")
print(f"  Most Mutated: {df.loc[df['mutation_pct'].idxmax(), 'layer']} ({df['mutation_pct'].max():.1f}%)")

In [ ]:
# Analyze function_id distribution changes
all_orig = []
all_evol = []

for layer_name in original_ids.keys():
    all_orig.extend(original_ids[layer_name])
    all_evol.extend(evolved_ids[layer_name])

orig_counts = Counter(all_orig)
evol_counts = Counter(all_evol)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Original distribution
axes[0].bar(orig_counts.keys(), orig_counts.values())
axes[0].set_xlabel('Function ID')
axes[0].set_ylabel('Count')
axes[0].set_title('Original Function ID Distribution')

# Evolved distribution
axes[1].bar(evol_counts.keys(), evol_counts.values(), color='orange')
axes[1].set_xlabel('Function ID')
axes[1].set_ylabel('Count')
axes[1].set_title('Evolved Function ID Distribution')

plt.tight_layout()
plt.show()

print(f"\n🔢 Function ID Usage:")
print(f"  Original - Most Common: {orig_counts.most_common(3)}")
print(f"  Evolved  - Most Common: {evol_counts.most_common(3)}")

In [ ]:
# Export results for later use
results = {
    'total_layers': len(original_ids),
    'total_neurons': df['total_neurons'].sum(),
    'total_mutations': int(df['mutations'].sum()),
    'mutation_rate': float(df['mutations'].sum() / df['total_neurons'].sum()),
    'per_layer_stats': mutation_stats,
    'function_id_distribution': {
        'original': dict(orig_counts),
        'evolved': dict(evol_counts)
    }
}

with open('evolution_analysis_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("💾 Results saved to evolution_analysis_results.json")

## 🎯 Key Findings

Run all cells to see:
- Which layers mutated most aggressively
- Function ID distribution shifts
- Visual mutation patterns across depth

**No GPU required!** 🎉